In [7]:
cd '/Users/bchnge/Documents/GitHub/home-credit-default-risk/'

/Users/bchnge/Documents/GitHub/home-credit-default-risk


In [13]:
from src import proj_utils, utils, classes

In [14]:
from importlib import reload

In [10]:
from pprint import pprint

In [11]:
from lightgbm.sklearn import LGBMClassifier

In [16]:
# Load data
df_train = proj_utils.load_data(data_path= '/Users/bchnge/data/home-credit-default-risk/',
                                train = True,
                                supp_dict = {'previous_application.csv.zip' : 'max',
                                             'credit_card_balance.csv.zip' : 'mean',
                                             'installments_payments.csv.zip' : 'min',
                                             'POS_CASH_balance.csv.zip' : 'mean',
                                             'bureau.csv.zip' : 'max'
                                            })

df_test = proj_utils.load_data(data_path = '/Users/bchnge/data/home-credit-default-risk/',
                               train = False,
                               supp_dict = {'previous_application.csv.zip' : 'max',
                                            'credit_card_balance.csv.zip' : 'mean',
                                            'installments_payments.csv.zip' : 'min',
                                            'POS_CASH_balance.csv.zip' : 'mean',
                                            'bureau.csv.zip' : 'max'
                                           })

Loading training applications
Loading previous_application.csv.zip
Loading credit_card_balance.csv.zip
Loading installments_payments.csv.zip
Loading POS_CASH_balance.csv.zip
Loading bureau.csv.zip
Loading test applications
Loading previous_application.csv.zip
Loading credit_card_balance.csv.zip
Loading installments_payments.csv.zip
Loading POS_CASH_balance.csv.zip
Loading bureau.csv.zip


In [39]:
reload(utils)
reload(classes)

<module 'src.classes' from '/Users/bchnge/Documents/GitHub/home-credit-default-risk/src/classes.py'>

In [40]:
data = classes.Dataset(df_train, df_test, 'TARGET')

...creating training matrix
...creating test matrix


In [52]:
data.X_train.columnsumns

Index(['NAME_PRODUCT_TYPE_x-sell_max', 'LIVINGAPARTMENTS_MEDI',
       'NAME_CASH_LOAN_PURPOSE_Education_mean',
       'NAME_CONTRACT_STATUS_Signed_4', 'NAME_CLIENT_TYPE_Repeater',
       'AMT_RECEIVABLE_PRINCIPAL_max', 'NAME_GOODS_CATEGORY_Tourism_mean',
       'NAME_CASH_LOAN_PURPOSE_Furniture_max',
       'NAME_CONTRACT_STATUS_Refused_max', 'DAYS_LAST_DUE_1ST_VERSION',
       ...
       'NAME_GOODS_CATEGORY_Medicine',
       'PRODUCT_COMBINATION_POS mobile with interest',
       'CODE_REJECT_REASON_SYSTEM_mean', 'AMT_BALANCE_mean',
       'CODE_REJECT_REASON_SYSTEM_max', 'NAME_CONTRACT_STATUS_Active',
       'CNT_DRAWINGS_ATM_CURRENT_max', 'AMT_PAYMENT_max', 'LANDAREA_MEDI',
       'NAME_TYPE_SUITE_Group of people_mean'],
      dtype='object', length=837)

In [46]:

# Clean and transform data

# Determine initial feature importances
data.ae_train_model(model = LGBMClassifier(n_estimators= 100, n_jobs=4))

In [48]:
# Auto-discover ratios weighted by feature importance
data.autoengineer_ratios(ae_params={'boosting_type': 'gbdt',
                                    'max_depth': -1,
                                    'objective': 'binary',
                                    'learning_rate': 0.0212,
                                    'reg_alpha': 0.8,
                                    'reg_lambda': 0.4,
                                    'subsample': 1,
                                    'feature_fraction': 0.3,
                                    'metric': 'auc',
                                    'random_state': 123,
                                    'n_estimators': 300, 
                                    'num_leaves': 40,
                                    'max_bin': 255,
                                    'min_data_in_leaf': 2400,
                                    'min_data_in_bin': 5},
                         n_iter = 10)

In [53]:
data.ae_discovery_ratios

[('DAYS_REGISTRATION', 'DAYS_LAST_DUE_1ST_VERSION_max', 1.0004759447305511),
 ('DAYS_INSTALMENT_mean', 'AMT_CREDIT_LIMIT_ACTUAL_mean', 1.0019457740250499)]

In [54]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline

In [58]:
from sklearn.preprocessing import FunctionTransformer

In [60]:
x = data.X_train.values

In [61]:
import numpy as np

In [65]:
x_ismissing

array([  5.35070290e-02,   6.83549532e-01,   5.35070290e-02,
         5.87523698e-02,   5.35070290e-02,   7.17392223e-01,
         5.35070290e-02,   5.35070290e-02,   5.35070290e-02,
         5.77247643e-02,   5.35070290e-02,   5.35070290e-02,
         0.00000000e+00,   1.43149351e-01,   2.11598935e-01,
         1.43149351e-01,   5.51791643e-01,   6.64977838e-01,
         5.35070290e-02,   5.35070290e-02,   5.87523698e-02,
         7.17392223e-01,   5.48565742e-02,   5.87523698e-02,
         5.35070290e-02,   5.35070290e-02,   5.35070290e-02,
         5.87523698e-02,   5.35070290e-02,   2.14626469e-03,
         7.17392223e-01,   8.01177844e-01,   1.43149351e-01,
         5.35070290e-02,   8.01177844e-01,   5.35070290e-02,
         5.35070290e-02,   1.43149351e-01,   5.48565742e-02,
         5.35070290e-02,   1.43149351e-01,   5.87523698e-02,
         1.98253071e-01,   5.35070290e-02,   7.17392223e-01,
         5.87523698e-02,   1.43149351e-01,   5.35070290e-02,
         5.35070290e-02,

In [76]:
FeatureReducer = FunctionTransformer(func = _feature_reducer, validate= False)

In [111]:
reload(utils)

<module 'src.utils' from '/Users/bchnge/Documents/GitHub/home-credit-default-risk/src/utils.py'>

In [115]:
reducer = utils.NanReducer()

In [116]:
a = reducer.fit_transform(data.X_train)

In [118]:
b = reducer.transform(data.X_test)

In [139]:
reload(classes)

<module 'src.classes' from '/Users/bchnge/Documents/GitHub/home-credit-default-risk/src/classes.py'>

In [120]:
from sklearn.preprocessing import Imputer

In [122]:
M1 = classes.Model(Pipeline([('impute', Imputer()),
                             ('clf', LogisticRegression())]),
          name = 'logistic regression',
          desc = 'logistic regression base')

M2 = classes.Model(LGBMClassifier(),
          name = "lgbm",
          desc = "lgbm")

M3 = classes.Model(Pipeline([('impute', Imputer()),
                             ('clf', GaussianNB())]),
          name = 'gaussian nb',
          desc = 'gnb')

models = [M1, M2, M3]

In [123]:
for m in models:
    print(m.name)
    m.train(data.X_train, data.y_train)

logistic regression
lgbm
gaussian nb


In [125]:
M1.validate_model(data.X_train, data.y_train)


In [126]:
M2.validate_model(data.X_train, data.y_train)
M3.validate_model(data.X_train, data.y_train)


In [128]:
M1_features = M2_features = M3_features = list(data.X_train.columns)

In [141]:
for m in models:
    print(m.validation_scores.mean())

0.640515328903
0.778392647709
0.611205219084


In [158]:
reload(classes)

<module 'src.classes' from '/Users/bchnge/Documents/GitHub/home-credit-default-risk/src/classes.py'>

In [159]:
mc = classes.ModelCollection([(M1, M1_features),
                              (M2, M2_features),
                              (M3, M3_features)])

In [160]:
mc.validate_model(data.X_train, data.y_train)

In [164]:
mc.validation_scores_stack.get('test_score').mean()

0.55456610917131322

In [166]:
from sklearn.feature_selection import SelectFromModel

In [167]:
mdl = LogisticRegression()

In [174]:
a = SelectFromModel(M1.clf, threshold = 0.25, prefit = True)

In [175]:
a.estimator

Pipeline(memory=None,
     steps=[('impute', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)), ('clf', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [180]:
piper = Pipeline([('impute', Imputer()),
                  ('lr', LogisticRegression(penalty = 'l1'))])

In [181]:
a = SelectFromModel(piper, threshold = 0.25)

In [183]:
from sklearn.feature_selection import chi2

In [ ]:
preprocesser = Pipeline([('imputer', Imputer()),
                         ('')])

In [189]:
data.X_train.describe()

,NAME_PRODUCT_TYPE_x-sell_max,LIVINGAPARTMENTS_MEDI,NAME_CASH_LOAN_PURPOSE_Education_mean,NAME_CONTRACT_STATUS_Signed_4,NAME_CLIENT_TYPE_Repeater,AMT_RECEIVABLE_PRINCIPAL_max,NAME_GOODS_CATEGORY_Tourism_mean,NAME_CASH_LOAN_PURPOSE_Furniture_max,NAME_CONTRACT_STATUS_Refused_max,DAYS_LAST_DUE_1ST_VERSION,...,NAME_GOODS_CATEGORY_Medicine,PRODUCT_COMBINATION_POS mobile with interest,CODE_REJECT_REASON_SYSTEM_mean,AMT_BALANCE_mean,CODE_REJECT_REASON_SYSTEM_max,NAME_CONTRACT_STATUS_Active,CNT_DRAWINGS_ATM_CURRENT_max,AMT_PAYMENT_max,LANDAREA_MEDI,NAME_TYPE_SUITE_Group of people_mean
count,291057.000000,97312.000000,291057.000000,289444.000000,291057.000000,8.690500e+04,291057.000000,291057.000000,291057.000000,289760.000000,...,291057.000000,291057.000000,291057.000000,86905.000000,291057.000000,86905.000000,61140.000000,2.916350e+05,124921.000000,291057.000000
mean,0.519785,0.101954,0.000782,0.000097,0.083746,1.379477e+05,0.001607,0.001907,0.344585,2312.237269,...,0.000959,0.057034,0.000336,71459.926952,0.001766,0.830999,3.995895,1.381792e+05,0.067169,0.001575
std,0.499609,0.093642,0.017387,0.009835,0.277008,1.660849e+05,0.033622,0.043626,0.475234,35478.680263,...,0.030946,0.231907,0.009591,109056.518226,0.041986,0.374755,3.639161,2.477880e+05,0.082167,0.026990
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,-2801.000000,...,0.000000,0.000000,0.000000,-2930.232558,0.000000,0.000000,0.000000,2.250000e-01,0.000000,0.000000
25%,0.000000,0.051300,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,-1999.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000,1.409717e+04,0.018700,0.000000
50%,1.000000,0.076100,0.000000,0.000000,0.000000,9.000000e+04,0.000000,0.000000,0.000000,-1089.000000,...,0.000000,0.000000,0.000000,26793.606316,0.000000,1.000000,3.000000,3.600000e+04,0.048700,0.000000
75%,1.000000,0.123100,0.000000,0.000000,0.000000,1.800000e+05,0.000000,0.000000,1.000000,-309.000000,...,0.000000,0.000000,0.000000,98610.502500,0.000000,1.000000,6.000000,1.329875e+05,0.086800,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.354829e+06,1.000000,1.000000,1.000000,365243.000000,...,1.000000,1.000000,1.000000,928686.323571,1.000000,1.000000,51.000000,3.771488e+06,1.000000,1.000000


In [188]:
a = chi2(Imputer().fit_transform(data.X_train), data.y_train)

ValueError: Input X must be non-negative.

In [182]:
a.transform(data.X_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [152]:
from mlxtend.classifier import StackingClassifier

In [154]:
a = StackingClassifier([M1, M2], LogisticRegression())

In [156]:
from sklearn.model_selection import cross_validate

In [ ]:
b = cross_validate(a, )

In [ ]:
#############################################################
models = {}
for m in M:
    # Define a model
    models[m] = {}

    # Tune parameters
    tuner = Tuner(m, data.X_train, data.y_train)
    for k in range(1,5):
        tuner.tune(kappa = k, pbounds = {}, n_iters = 10)
    

# Compare and stack models
mc = ModelCollection(models)